In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./test_y_pred.csv
./train.csv
./test_x.csv
./text_Wu.ipynb
./.ipynb_checkpoints/text_Wu-checkpoint.ipynb


In [2]:
train = pd.read_csv("train.csv")
test_x = pd.read_csv("test_x.csv")
train

,id,keyword,location,text,target
0,2115,catastrophe,Florida,@deb117 7/30 that catastrophe man opens school...,0
1,9232,suicide%20bombing,NaN,meek mill should join isis since he loves suic...,0
2,8131,rescued,Ireland,Three beached whales rescued in Kerry - http:/...,1
3,8236,riot,Seattle,Southeast Dirt Riot Series Crowns Champions: ...,0
4,9822,trauma,NaN,Hiroshima: They told me to paint my story: Eig...,1
...,...,...,...,...,...
5324,6371,hostages,Germany,@banditregina I also loved the episode 'Bang' ...,0
5325,9388,survived,"Austin, TX",Survived Spanish!! @ Sweet Ritual https://t.co...,0
5326,5366,fire%20truck,United States,#RFP: Fire Truck Service Body for F-450 (Fire ...,0
5327,479,armageddon,NaN,So the Ahamedis think the Messiah had already ...,0


In [3]:
import re

# Example: URL removal
def remove_url(data):
    re1 = r"http://\S+|www\.\S+"
    re2 = r"https://\S+|www\.\S+"
    url_clean= re.compile("(%s|%s)" % (re1, re2))
    data=url_clean.sub(r'',data)
    return data

# !!! YOUR TASK 1 (4 point): add TWO more preprocessing steps from the above list
# you can choose two functions to implement
def to_lowercase(data):
    # your implementation
    
    return data.lower()

def remove_hashtag(data):
    # your implementation
    # Define a regular expression pattern to match hashtags
    hashtag_pattern =r'#\w+'
    
    # Use re.sub() to replace hashtags with an empty string
    data = re.sub(hashtag_pattern, '', data)
    return data

In [4]:
#see the preprocess result
print('before removal: ', train.iloc[5326]['text'])

#train['text'] = train['text'].apply(remove_url)
#test_x['text'] = test_x['text'].apply(remove_url)
train['text'] = train['text'].apply(lambda x: remove_url(to_lowercase(remove_hashtag(x))))
test_x['text'] = test_x['text'].apply(lambda x: remove_url(to_lowercase(remove_hashtag(x))))

print('after removal: ', train.iloc[5326]['text'])
# proprocess both test and train data, result looks good , 5326 of test lower casem, no # anymore 

before removal:  #RFP: Fire Truck Service Body for F-450 (Fire fighting rescue &amp; safety equipment Transporta... http://t.co/8GtRvEcE1N
after removal:  : fire truck service body for f-450 (fire fighting rescue &amp; safety equipment transporta... 


In [5]:
# This package generates bag of words feature for each text
from sklearn.feature_extraction.text import CountVectorizer 

# doc for this function: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer(stop_words='english', max_df=0.9, min_df=2, max_features=5000) 

# use training data to construct vocabulary and turn training text into BOW vectors
train_bow = vectorizer.fit_transform(train['text'])
vocab = vectorizer.vocabulary_.copy()
# You can change the vocab size by tuning "max_df", "min_df" or "max_features" in "CountVectorizer"
print('vocab size: ', len(vocab))  
print (train_bow.shape) # num_train_text * vocab_size

# !!! YOUR TASK 2 (1 point): transform test text into BOW vectors using vectorizer
test_bow = vectorizer.transform(test_x['text'])
print (test_bow.shape) # num_train_text * vocab_size

vocab size:  5000
(5329, 5000)
(2284, 5000)


In [6]:
# Use basic ML models on BOW features
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Split given training data into training and validation set
X_train,X_val,y_train,y_val = train_test_split(train_bow, train['target'], test_size=0.3, random_state=11)

LR = RandomForestClassifier()
#tuning the parameters of random forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    
}


#Grid search
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)


{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}


In [7]:
LR = RandomForestClassifier(n_estimators=300, max_depth=None, min_samples_leaf=1, min_samples_split=10)
LR.fit(X_train.toarray(),y_train)
print('training recall: ', recall_score(y_train, LR.predict(X_train.toarray())))
print('validation recall: ', recall_score(y_val, LR.predict(X_val.toarray())))
test_y_pred = LR.predict(test_bow)

predictions_df = pd.DataFrame({
    'id': test_x['id'],
    'target': test_y_pred
})
predictions_df.to_csv("test_y_pred.csv", index=False)


# Optional: apply more advanced model selection technique
# e.g., sklearn.model_selection.GridSearchCV: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html



training recall:  0.9146417445482866
validation recall:  0.6394658753709199


In [7]:
# train word embeddings using gensim library

from gensim.models import Word2Vec

train_docs = [train['text'].iloc[i].split(' ') for i in range(len(train['text']))] # gensim only takes list of word list as `sentences`

w2v_model = Word2Vec(sentences=train_docs, vector_size=100, window=5, negative=20, min_count=2, workers=1)
w2v_model.train(train_docs, total_examples=len(train_docs), epochs=10)


(526856, 795450)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [9]:
# check the learned word embeddings

# vocabulary
w2v_vocav = w2v_model.wv.index_to_key
print('example words in vocab: ', [w2v_vocav[i] for i in range(200)])

# example embedding
example_word = 'bomb'
word_vec = w2v_model.wv[example_word]
print('learned embedding of bomb: ', word_vec)

# !!! YOUR TASK (2 point): show the top-10 words most similar to 'bomb'
# you may calculate manually, or refer to gensim function: https://radimrehurek.com/gensim/models/word2vec.html
similar_words = w2v_model.wv.most_similar('bomb', topn=10)

# Print the similar words
for word, score in similar_words:
    print(f'{word}: {score}')

# !!! Your TASK (1 point): tune the arguments of Word2Vec to obtain different word embeddings 
# 
# Tune the parameters of Word2Vec, I will come back to this task later so that the work folw is not interrupted now.
w2v_model = Word2Vec(sentences=train_docs, vector_size=200, window=10, negative=20, min_count=3, workers=1)
w2v_model.train(train_docs, total_examples=len(train_docs), epochs=20)
w2v_vocav = w2v_model.wv.index_to_key
print('example words in vocab: ', [w2v_vocav[i] for i in range(200)])
example_word = 'bomb'
word_vec = w2v_model.wv[example_word]
print('learned embedding of bomb: ', word_vec)

example words in vocab:  ['', 'the', 'a', 'to', 'in', 'of', 'and', 'i', 'is', 'for', 'on', 'you', '-', 'my', 'that', 'with', 'by', 'at', 'it', 'this', 'are', 'from', 'be', 'was', 'have', 'like', 'as', 'just', 'up', 'so', 'but', '&amp;', 'your', 'not', 'me', 'will', 'after', 'an', 'has', 'out', "i'm", 'when', 'get', 'all', '??', 'fire', 'if', 'new', 'via', 'no', 'more', 'we', 'or', 'about', 'he', 'how', 'they', 'over', 'what', 'into', "it's", 'now', 'people', 'do', 'one', '...', 'who', 'been', 'can', '|', "don't", 'than', '2', 'emergency', 'some', 'there', 'his', 'police', 'burning', 'disaster', 'still', 'her', 'body', 'would', ':', 'love', 'suicide', 'got', 'video', 'going', 'why', 'man', 'back', 'had', 'killed', 'see', 'first', 'off', 'california', 'us', 'buildings', 'storm', 'full', '3', 'families', "can't", '????', 'car', 'know', 'nuclear', 'were', 'two', 'train', 'our', 'day', 'crash', 'time', 'say', 'go', 'think', '@youtube', 'them', 'news', 'last', 'could', 'many', 'their', 'bomb

In [10]:
# Use basic ML models on word embeddings

# !!! YOUR TASK (2 points): turn word embedding to doc embedding
# Choice 1 (as shown below): manualy take the average of word embeddings in each doc
# Choise 2: gensim doc2vec: https://radimrehurek.com/gensim/models/doc2vec.html 
def doc2vec(docs, w2v_model):
    w2v = []
    for doc in docs:
        word_embs = [w2v_model.wv[w] for w in doc if w in w2v_model.wv] # get a list of word embeddings
        
        # TODO: take (weighted) average to get doc embedding
        doc_emb = np.mean(np.array(word_embs), axis=0) 
        
        # filling zeros for empty doc
        if len(word_embs) == 0:
            doc_emb = [0 for i in range(w2v_model.vector_size)]
        
        w2v.append(doc_emb)
    
    return np.array(w2v)
        
# transform training tweets into embeddings
train_w2v = doc2vec(train_docs, w2v_model)
print('training word2vec shape: ', train_w2v.shape)

test_w2v = doc2vec(test_x['text'], w2v_model) 
print('test word2vec shape: ', test_w2v.shape)

        
# transform training tweets into embeddings
train_w2v = doc2vec(train_docs, w2v_model)
print('training word2vec shape: ', train_w2v.shape)

test_w2v = doc2vec(test_x['text'], w2v_model) 
print('test word2vec shape: ', test_w2v.shape)

# !!! YOUR TASK (2 point): apply a basic ML model on word embeddings
# you may consider tune the word2vec model to gain better embeddings (refer to gensim document about argument details: https://radimrehurek.com/gensim/models/word2vec.html)
X_train,X_val,y_train,y_val = train_test_split(train_w2v, train['target'], test_size=0.3, random_state=11)
# Initialize and train a logistic regression classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = clf.predict(X_val)

# Calculate accuracy on the validation set
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

test_y_pred = clf.predict(test_w2v)

predictions_df = pd.DataFrame({
    'id': test_x['id'],
    'target': test_y_pred
})
predictions_df.to_csv("test_y_pred.csv", index=False)


training word2vec shape:  (5329, 200)


/Users/siyuwu/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/siyuwu/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


test word2vec shape:  (2284, 200)
training word2vec shape:  (5329, 200)
test word2vec shape:  (2284, 200)
Validation accuracy: 76.11%


/Users/siyuwu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
